<a href="https://colab.research.google.com/github/heyaug/homework/blob/master/l2%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import math
import operator
import pandas as pd

In [0]:
file_path = "user_taggedbookmarks-timestamps.dat"

In [0]:
def load_data():
    print("开始数据加载...")
    df = pd.read_csv(file_path, sep='\t')
    print("数据集大小为 %d." % (len(df.index)))
    print("设置tag的人数 %d." % (len(df.userID.unique())))
    print("数据加载完成\n")
    return df

In [0]:
data=load_data()

开始数据加载...
数据集大小为 437593.
设置tag的人数 1867.
数据加载完成



In [0]:
def train_test_split(df,ratio, seed=100):
    random.seed(seed)
    # ratio比例设置为测试集,从df的index中随机取测试集的index
    test_index=random.sample(list(df.index), int(len(df.index)*ratio))
    test_data=pd.DataFrame(df,index=test_index)
    # 训练集的index是df的index-测试集的index
    train_index=set(df.index)-set(test_index)
    train_data=pd.DataFrame(df,index=train_index)
    print("训练集样本数 %d, 测试集样本数 %d" % (len(train_data.userID.unique()),len(test_data.userID.unique())))
    return test_data,train_data

In [0]:
test_data,train_data=train_test_split(data,0.2, seed=100)

训练集样本数 1861, 测试集样本数 1825


In [0]:
# 统一格式
def transform(df):
    records={}
    for user,subdf in df.groupby(["userID"]):
        items={}
        for item,subdf2 in subdf.groupby("bookmarkID"):
            items.update({item:list(subdf2.tagID.values)})
        records.update({user:items})
    return records

In [0]:
def initStat(df):
    user_items={}
    user_tags={}
    # 从userid的groupby中获取user_items和user_tags
    for user,subdf in df.groupby(["userID"]):
        # 用户使用同一商品的次数
        user_items.update({user:dict(subdf.bookmarkID.value_counts().items())})
        # 用户使用同一标签的次数
        user_tags.update({user:dict(subdf.tagID.value_counts().items())})
    tag_items={}
    for item,subdf in df.groupby(["bookmarkID"]):
        # 商品打过某一标签的次数
        tag_items.update({item:dict(subdf.tagID.value_counts().items())})
    print("user_tags, tag_items, user_items初始化完成.")
    print("user_tags大小 %d, tag_items大小 %d, user_items大小 %d" % (len(user_tags), len(tag_items), len(user_items)))
    return user_items,user_tags,tag_items

In [0]:
user_items,user_tags,tag_items=initStat(train_data)

user_tags, tag_items, user_items初始化完成.
user_tags大小 1861, tag_items大小 67469, user_items大小 1861


In [0]:
# 字典格式
test_data=transform(test_data)
train_data=transform(train_data)

In [0]:
# 推荐商品计算分数，用户对某商品的分数=用户对某标签的使用次数*某商品被打上该标签的次数
def recommend(user,N):
    result=[]
    # 用户未使用过的商品
    items=list(set(tag_items.keys())-{i[0] for i in list(user_items[user].items())})
    # 标签 和 用户使用该标签的次数
    for tag,u_used_tag_times in list(user_tags[user].items()):
        for item in items:
            # 某商品被打某标签的次数
            i_used_tag_times=tag_items.get(item).get(tag)
            if i_used_tag_times:
                result.append((item,u_used_tag_times*i_used_tag_times))
    # 根据分数倒序排序
    return sorted(result,key=lambda x:x[1],reverse=True)[:N]

In [0]:
recommend(8,5)

[(1526, 81), (918, 54), (1416, 54), (4639, 54), (6656, 54)]

In [0]:
def precisionAndRecall(N):
    hit = 0
    h_recall = 0
    h_precision = 0
    for user,items in test_data.items():
        if user not in train_data:
            continue
        # 获取Top-N推荐列表
        rank = recommend(user, N)
        for item,rui in rank:
            if item in items:
                hit = hit + 1
        h_recall = h_recall + len(items)
        h_precision = h_precision + N
    #print('一共命中 %d 个, 一共推荐 %d 个, 用户设置tag总数 %d 个' %(hit, h_precision, h_recall))
    # 返回准确率 和 召回率
    return (hit/(h_precision*1.0)), (hit/(h_recall*1.0))

In [0]:
precisionAndRecall(5)